# Предсказание стоимости жилья

## Описание

В проекте нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Задание необходимо выполнить используя **PySpark**.

Задача - построить две модели линейной регрессии на разных наборах данных:
- используя все данные из файла;
- используя только числовые переменные, исключив категориальные

Оценки качества моделей провести по метрикам **RMSE, MAE и R2**.

## Подготовка данных

### Импорт библиотек и модулей

In [1]:
import pandas as pd 
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder, Imputer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

RND_ST = 42
N_FOLDS = 5

In [2]:
pyspark.__version__

'3.0.2'

### Инициация spark сессии, загрузка данных

In [3]:
spark = (SparkSession
         .builder
         .master('local')
         .appName('housingPrice_linearRegression')
         .getOrCreate()
        )

In [4]:
df_sprk = spark.read.load('/datasets/housing.csv',
                         format='csv',
                         sep=',',
                         inferSchema=True,
                         header='true')

### Просмотр данных, заполнение пропусков, проверка на дубликаты

In [5]:
df_sprk.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [8]:
(df_sprk.describe().select('summary',
                           F.col('longitude').cast(DecimalType(7,1)),
                           F.col('latitude').cast(DecimalType(7,1)),
                           F.col('housing_median_age').cast(DecimalType(7,1)),
                           F.col('total_rooms').cast(DecimalType(7,1)),
                           F.col('total_bedrooms').cast(DecimalType(7,1)),
                           F.col('population').cast(DecimalType(7,1)),
                           F.col('households').cast(DecimalType(7,1)),
                           F.col('median_income').cast(DecimalType(7,1)),
                           F.col('median_house_value').cast(DecimalType(7,1)),
                           'ocean_proximity'
                          )
 .toPandas()
 .style.highlight_min(color='coral', subset=df_sprk.columns[1:-1], axis=1) # подсвечиваем мин. значение в строках
)

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640.0,20640.0,20640.0,20640.0,20433.0,20640.0,20640.0,20640.0,20640.0,20640
1,mean,-119.6,35.6,28.6,2635.8,537.9,1425.5,499.5,3.9,206855.8,None
2,stddev,2.0,2.1,12.6,2181.6,421.4,1132.5,382.3,1.9,115395.6,None
3,min,-124.4,32.5,1.0,2.0,1.0,3.0,1.0,0.5,14999.0,<1H OCEAN
4,max,-114.3,42.0,52.0,39320.0,6445.0,35682.0,6082.0,15.0,500001.0,NEAR OCEAN


Видим, что пропуски есть в столбце `total_bedrooms`. Проверим на пропуски все столбцы средствами PySpark

In [9]:
for c in df_sprk.columns:
    nan_qty = df_sprk.select(c).filter( F.isnull(c) | F.isnan(c) ).count()
    print( c, nan_qty )

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


#### Восстановление пропусков с применением `pyspark.ml.feature.Imputer` 

In [10]:
df_sprk.select('total_rooms','total_bedrooms').filter(F.isnull('total_bedrooms')).show(7)

+-----------+--------------+
|total_rooms|total_bedrooms|
+-----------+--------------+
|     1256.0|          null|
|      992.0|          null|
|     5154.0|          null|
|      891.0|          null|
|      746.0|          null|
|     3342.0|          null|
|     3759.0|          null|
+-----------+--------------+
only showing top 7 rows



In [11]:
imputer = Imputer(inputCol='total_bedrooms',
                  outputCol='total_bedrooms_out',
                  strategy='median')

In [12]:
model = imputer.fit(df_sprk)
df_imp = model.transform(df_sprk)
df_imp.select('total_rooms','total_bedrooms','total_bedrooms_out').filter(F.isnull('total_bedrooms')).show(7)

+-----------+--------------+------------------+
|total_rooms|total_bedrooms|total_bedrooms_out|
+-----------+--------------+------------------+
|     1256.0|          null|             435.0|
|      992.0|          null|             435.0|
|     5154.0|          null|             435.0|
|      891.0|          null|             435.0|
|      746.0|          null|             435.0|
|     3342.0|          null|             435.0|
|     3759.0|          null|             435.0|
+-----------+--------------+------------------+
only showing top 7 rows



Не самый лучший вариант ... все значения одинаковые и равны медианному значению в столбце `total_bedrooms`

#### Восстановление пропусков с применением `sklearn.impute.IterativeImputer`

In [13]:
df_pnd = df_sprk.toPandas()

In [14]:
col = df_pnd.columns.drop(['ocean_proximity', 'median_house_value'])

In [15]:
df_pnd.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [16]:
na_indx = df_pnd[df_pnd['total_bedrooms'].isna()].index

In [17]:
itrImp = IterativeImputer(
    estimator = BayesianRidge(),       # модель предсказания значений
    max_iter=20,                       # макс. число итераций
    n_nearest_features=col.shape[0]-1, # число признаков, учитываемых при восстановлении значения
    min_value=1,                       # мин. число возвращаемых значений
    random_state=RND_ST)

In [18]:
itrImp.fit(df_pnd[col])

IterativeImputer(estimator=BayesianRidge(), max_iter=20, min_value=1,
                 n_nearest_features=7, random_state=42)

In [19]:
df_pnd_itrImp = pd.DataFrame(data=itrImp.transform(df_pnd[col]), columns=col)
df_pnd_itrImp['ocean_proximity'] = df_pnd['ocean_proximity']
df_pnd_itrImp['median_house_value'] = df_pnd['median_house_value']
df_pnd_itrImp['total_bedrooms'] = round(df_pnd_itrImp['total_bedrooms'])

In [20]:
df_pnd_itrImp.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
ocean_proximity       0
median_house_value    0
dtype: int64

In [21]:
df_pnd_itrImp.loc[na_indx, :]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
290,-122.16,37.77,47.0,1256.0,219.0,570.0,218.0,4.3750,NEAR BAY,161900.0
341,-122.17,37.75,38.0,992.0,282.0,732.0,259.0,1.6196,NEAR BAY,85100.0
538,-122.28,37.78,29.0,5154.0,1283.0,3741.0,1273.0,2.5762,NEAR BAY,173400.0
563,-122.24,37.75,45.0,891.0,132.0,384.0,146.0,4.9489,NEAR BAY,247100.0
696,-122.10,37.69,41.0,746.0,157.0,387.0,161.0,3.9063,NEAR BAY,178400.0
...,...,...,...,...,...,...,...,...,...,...
20267,-119.19,34.20,18.0,3620.0,781.0,3171.0,779.0,3.3409,NEAR OCEAN,220500.0
20268,-119.18,34.19,19.0,2393.0,775.0,1938.0,762.0,1.6953,NEAR OCEAN,167400.0
20372,-118.88,34.17,15.0,4260.0,754.0,1701.0,669.0,5.1033,<1H OCEAN,410700.0
20460,-118.75,34.29,17.0,5512.0,886.0,2734.0,814.0,6.6073,<1H OCEAN,258100.0


In [22]:
df_itrImp = spark.createDataFrame(df_pnd_itrImp)

In [23]:
df_itrImp.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- median_house_value: double (nullable = true)



Восстановление пропусков с применением класса `sklearn.impute.IterativeImputer` учитывает зависимость от значений других признаков, что на мой взгляд более предпочтительно.

#### Проверка на дубликаты

In [24]:
df_dupl=df_itrImp.groupBy(df_itrImp.columns).count().filter('count > 1')
df_dupl.drop('count').show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+---------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|ocean_proximity|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+---------------+------------------+
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+---------------+------------------+



Дубликатов нет

#### Вывод

Пропуски восстановлены с применением класса `sklearn.impute.IterativeImputer` (преобразование из spark.DatFrame в pandas.Dataframe и после восстановления - обратно). Дубликаты отсутствуют.

### Трансформация признаков

Разделение на категориальные, числовые и целевой признак

In [25]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ['longitude',
                   'latitude',
                   'housing_median_age',
                   'total_rooms',
                   'total_bedrooms',
                   'population', 
                   'households',
                   'median_income']
target = 'median_house_value'

#### Категориальные признаки

К категориальным строковым столбцам применяем трансформер `StringIndexer`

In [26]:
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols])

df_catIdx = indexer.fit(df_itrImp).transform(df_itrImp)

In [27]:
col2show = [c for c in df_catIdx.columns for i in categorical_cols if (c.startswith(i))]

df_catIdx.select(col2show).sample(fraction=.001, seed=RND_ST).show()

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|      <1H OCEAN|                0.0|
|      <1H OCEAN|                0.0|
|      <1H OCEAN|                0.0|
|     NEAR OCEAN|                2.0|
|      <1H OCEAN|                0.0|
|         INLAND|                1.0|
|      <1H OCEAN|                0.0|
|      <1H OCEAN|                0.0|
|      <1H OCEAN|                0.0|
|      <1H OCEAN|                0.0|
|         INLAND|                1.0|
|         INLAND|                1.0|
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
|         INLAND|                1.0|
|     NEAR OCEAN|                2.0|
|      <1H OCEAN|                0.0|
|      <1H OCEAN|                0.0|
+---------------+-------------------+
only showing top 20 rows



Применим **OHE** кодирование к категориальному столбцу `ocean_proximity_idx`

In [28]:
OHEencoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                           outputCols=[c+'_ohe' for c in categorical_cols])

df_ohe = OHEencoder.fit(df_catIdx).transform(df_catIdx)

In [29]:
col2show = [c for c in df_ohe.columns for i in categorical_cols if (c.startswith(i))]

df_ohe.select(col2show).sample(fraction=.001, seed=RND_ST).show()

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|      <1H OCEAN|                0.0|      (4,[0],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCE

Объединим категориальные признаки в вектор, применив `VectorAssembler`

In [30]:
cat_assembler = VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                outputCol='cat_features')

df_catVect = cat_assembler.transform(df_ohe).drop('ocean_proximity',
                                                  'ocean_proximity_idx',
                                                  'ocean_proximity_ohe')

In [31]:
df_catVect.select(df_catVect.columns[-3:]).show(7)

+-------------+------------------+-------------+
|median_income|median_house_value| cat_features|
+-------------+------------------+-------------+
|       8.3252|          452600.0|(4,[3],[1.0])|
|       8.3014|          358500.0|(4,[3],[1.0])|
|       7.2574|          352100.0|(4,[3],[1.0])|
|       5.6431|          341300.0|(4,[3],[1.0])|
|       3.8462|          342200.0|(4,[3],[1.0])|
|       4.0368|          269700.0|(4,[3],[1.0])|
|       3.6591|          299200.0|(4,[3],[1.0])|
+-------------+------------------+-------------+
only showing top 7 rows



#### Числовые признаки

Собираем все числовые признаки (список `numerical_cols`) в вектор `num_features`, применяя класс `VectorAssembler`

In [32]:
num_assembler = VectorAssembler(inputCols=numerical_cols, outputCol='num_features')

df_catNumVect = num_assembler.transform(df_catVect)

Масштабируем значения в векторе `num_features` классом `StandardScaler`, сохраняя в новый вектор `num_features_scl`

In [33]:
standardScaler = StandardScaler(inputCol='num_features', outputCol='num_features_scl')

df_catNumSclVect = standardScaler.fit(df_catNumVect).transform(df_catNumVect).drop('num_features')

#### Объединение признаков

Объединяем в единый вектор `united_features` векторы категориальных и числовых признаков, применяя класс `VectorAssembler`

In [34]:
catNum_features = ['cat_features', 'num_features_scl']

final_assembler = VectorAssembler(inputCols=catNum_features, outputCol='united_features') 

df_unitVect = final_assembler.transform(df_catNumSclVect)

df_unitVect.select(df_unitVect.columns[-4:]).sample(fraction=.0007, seed=RND_ST).show()

+------------------+-------------+--------------------+--------------------+
|median_house_value| cat_features|    num_features_scl|     united_features|
+------------------+-------------+--------------------+--------------------+
|          121900.0|(4,[0],[1.0])|[-59.020777466540...|[1.0,0.0,0.0,0.0,...|
|          250000.0|(4,[2],[1.0])|[-58.970865603989...|[0.0,0.0,1.0,0.0,...|
|          285300.0|(4,[0],[1.0])|[-59.100636446621...|[1.0,0.0,0.0,0.0,...|
|           72400.0|(4,[1],[1.0])|[-60.138803187682...|[0.0,1.0,0.0,0.0,...|
|          193800.0|(4,[0],[1.0])|[-58.851077133866...|[1.0,0.0,0.0,0.0,...|
|          132800.0|(4,[0],[1.0])|[-58.910971368928...|[1.0,0.0,0.0,0.0,...|
|          109100.0|(4,[1],[1.0])|[-60.538098088090...|[0.0,1.0,0.0,0.0,...|
|           73500.0|(4,[2],[1.0])|[-58.451782233459...|[0.0,0.0,1.0,0.0,...|
|          144000.0|(4,[2],[1.0])|[-58.416843929673...|[0.0,0.0,1.0,0.0,...|
|          111500.0|(4,[2],[1.0])|[-60.118838442662...|[0.0,0.0,1.0,0.0,...|

#### Вывод

Сформированы векторы признаков:
- вектор категориальных признаков
- вектор числовых признаков с масштабированными значениями
- вектор общих (объединенных) признаков

### Разделение на выборки

Для тренировочных и тестовых данных отберем 80% и 20% данных соответственно

In [35]:
df_train, df_test = df_unitVect.randomSplit([.8,.2], seed=RND_ST)
print(f'Тренировочные данные: {df_train.count()} строк \nТестовые данные: {df_test.count()} строк')

Тренировочные данные: 16560 строк 
Тестовые данные: 4080 строк


## Подбор гиперпараметров

Класс `LinearRegression` имеет два основных гиперпараметра:
- regParam
- elasticNetParam 

Эти параметры задают вид регуляризации: L1(Lasso), L2(Ridge) или Elastic Net.

Таким образом, задавая определённые значения regParam и elasticNetParam, можно получить следующие результаты:
- `regParam = 0` – это линейная регрессия **без регуляризации**;
- `regParam > 0`, `elasticNetParam = 0` ведет к **L2**-регуляризация (Ridge);
- `regParam > 0`, `elasticNetParam = 1` ведет к **L1**-регуляризация (Lasso);
- `regParam > 0`, `0 < elasticNetParam < 1` ведет к **L1 + L2** (Elastic Net).

Подбор будем проводить кросс-валидацией на тренировочных данных. В PySpark кросс-валидация реализуется через `CrossValidator()`. Нам нужно указать четыре аргумента:

- estimator — модель Machine Learning, в нашем случае `LinearRegression()`;
- estimatorParamMaps — алгоритм оптимизации гиперпараметров, в нашем случае GridSearch применяя класс `ParamGridBuilder()`;
- evaluator — метрика качества, в нашем случае `RegressionEvaluator()`;
- количество фолдов, в нашем случае `N_FOLDS`.

Оценку проводим по средней абсолютной ошибке `MAE`.

In [36]:
lnr_reg = LinearRegression(featuresCol='united_features', labelCol=target)

In [37]:
lnr_reg_GridSearch = (ParamGridBuilder()
                     .addGrid(lnr_reg.regParam, [0, .1, .5, 1, 2])
                     .addGrid(lnr_reg.elasticNetParam, [0, .5, 1])
                     .build())

lnr_reg_Evaluator = RegressionEvaluator(predictionCol='prediction',
                                        labelCol=target,
                                        metricName='mae')

lnr_reg_CV = CrossValidator(estimator=lnr_reg,
                            estimatorParamMaps=lnr_reg_GridSearch,
                            evaluator=lnr_reg_Evaluator,
                            numFolds=N_FOLDS, 
                            seed=RND_ST)

In [39]:
lnr_reg_CVModels = lnr_reg_CV.fit(df_train)

Выведем значения ключевых метрик лучшей модели:

In [40]:
lnr_reg_CVSummary = lnr_reg_CVModels.bestModel.summary
print('Средняя абсолютная ошибка (MAE):', "{:.3f}".format(lnr_reg_CVSummary.meanAbsoluteError))
print('Среднеквадратичная ошибка (RMSE):', "{:.3f}".format(lnr_reg_CVSummary.rootMeanSquaredError))
print('Коэффициент детерминации R2:', "{:.3f}".format(lnr_reg_CVSummary.r2))

Средняя абсолютная ошибка (MAE): 49424.293
Среднеквадратичная ошибка (RMSE): 68179.687
Коэффициент детерминации R2: 0.647


Выведем значения параметров `elasticNetParam` и `regParam` лучшей модели:

In [41]:
print('Значение regParam', lnr_reg_CVModels.bestModel.getRegParam())
print('Значение elasticNetParam', lnr_reg_CVModels.bestModel.getElasticNetParam())

Значение regParam 0.0
Значение elasticNetParam 0.0


### Вывод

Лучшая модель получилась без регуляризации (`regParam = 0`). Наиболее легко интерпретируемые метрики:
- MAE чуть более 49.4K т.е. в среднем значение, предсказанное моделью отличается от истинного на +/- 49.4K
- R2 .647 т.е. почти 65% дисперсии целевой переменной объясняется моделью

## Обучение модели

Теперь надо обучить модель на всех данных (столбец `united_features`) и только на числовых данных (столбец `num_features_scl`)

Гиперпараметры в обоих случаях:
- `regParam=0`
- `elasticNetParam=0`

### Обучение и прогноз на всех данных (числовые + категориальные)

In [42]:
model = LinearRegression( featuresCol='united_features', labelCol=target, regParam=0, elasticNetParam=0 )
model_allData = model.fit(df_train)

In [43]:
allPredictions = model_allData.transform(df_test)

In [44]:
predictedAllValues = allPredictions.select('median_house_value', 'prediction')
predictedAllValues.show(15)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0| 150527.4692572886|
|          106700.0|216773.39134276053|
|           73200.0|126057.29608237231|
|           90100.0|194379.38182429038|
|           67000.0|152638.51120139612|
|           86400.0| 186001.1827696492|
|           70500.0|162426.99246193655|
|           85100.0|179009.06698814826|
|           80500.0| 181297.0111426036|
|           96000.0|170098.58287173184|
|           75500.0|135849.94067784166|
|           75000.0|104004.19391255407|
|          100600.0|189557.96341805533|
|           74100.0| 156033.7483477085|
|           66800.0|133502.30632839398|
+------------------+------------------+
only showing top 15 rows



Выведем метрики:

In [45]:
evaluator = RegressionEvaluator()
evaluator.setPredictionCol('prediction')
evaluator.setLabelCol(target)

RegressionEvaluator_169928090d22

In [46]:
all_data_metrics = ['{:.0f}'.format(evaluator.evaluate(predictedAllValues, {evaluator.metricName: 'mae'})),
                    '{:.0f}'.format(evaluator.evaluate(predictedAllValues, {evaluator.metricName: 'rmse'})),
                    '{:.3f}'.format(evaluator.evaluate(predictedAllValues, {evaluator.metricName: 'r2'}))]


print('Средняя абсолютная ошибка (MAE):', all_data_metrics[0])
print('Среднеквадратичная ошибка (RMSE):', all_data_metrics[1])
print('Коэффициент детерминации R2:', all_data_metrics[2])

Средняя абсолютная ошибка (MAE): 50774
Среднеквадратичная ошибка (RMSE): 70588
Коэффициент детерминации R2: 0.640


### Обучение и прогноз используя только числовые данные

In [47]:
model = LinearRegression( featuresCol='num_features_scl', labelCol=target, regParam=0, elasticNetParam=0 )
model_numData = model.fit(df_train)

numPredictions = model_numData.transform(df_test)

predictedNumValues = numPredictions.select('median_house_value', 'prediction')
predictedNumValues.show(15)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|101342.05023674667|
|          106700.0|189983.06552391686|
|           73200.0| 76358.54149507266|
|           90100.0|161923.11289616209|
|           67000.0|120158.91067604627|
|           86400.0|156092.36656785803|
|           70500.0|129762.65938221896|
|           85100.0| 149549.9846050418|
|           80500.0|150075.94286684692|
|           96000.0| 133900.9546660469|
|           75500.0| 98132.71619331138|
|           75000.0| 50419.29025894124|
|          100600.0|156201.01114308322|
|           74100.0|123191.17215683032|
|           66800.0|102375.65871356381|
+------------------+------------------+
only showing top 15 rows



In [48]:
num_data_metrics = ['{:.0f}'.format(evaluator.evaluate(predictedNumValues, {evaluator.metricName: 'mae'})),
                    '{:.0f}'.format(evaluator.evaluate(predictedNumValues, {evaluator.metricName: 'rmse'})),
                    '{:.3f}'.format(evaluator.evaluate(predictedNumValues, {evaluator.metricName: 'r2'}))]


print('Средняя абсолютная ошибка (MAE):', num_data_metrics[0])
print('Среднеквадратичная ошибка (RMSE):', num_data_metrics[1])
print('Коэффициент детерминации R2:', num_data_metrics[2])

Средняя абсолютная ошибка (MAE): 51669
Среднеквадратичная ошибка (RMSE): 71548
Коэффициент детерминации R2: 0.630


## Анализ результатов

Выведем в таблице метрики работы 1-й и 2-й моделей, а также величину изменения метрик 2-й модели относительно 1-й:

In [51]:
pd.DataFrame(data=[ all_data_metrics,
                   num_data_metrics,
                   [float(a[1]) - float(a[0]) for a in list(zip(all_data_metrics, num_data_metrics))] ],
             columns=['MAE', 'RMSE', 'R2'],
             index=['NUM + CAT data', 'NUM data', 'Изменение'])

,MAE,RMSE,R2
NUM + CAT data,50774,70588,0.640
NUM data,51669,71548,0.630
Изменение,895.0,960.0,-0.01


### Вывод

Ожидаемо метрики работы модели на ограниченном объеме данных хуже.
- средняя абсолютная ошибка увеличилась на 895
- среднеквадратичная ошибка увеличилась на 960
- коэффициент детерминации R2 снизился на 1 процентный пункт - т.о. именно такую часть дисперсии в данных объясняют категориальные данные

In [52]:
spark.stop()